In [ ]:
import fxcmpy
import time
import numpy as np
import pandas as pd
import threading as thr
import re
from datetime import datetime

In [ ]:
TOKEN='4009416fe3d78b0ad8404d1d183cacffb4245475'
con = fxcmpy.fxcmpy(access_token=TOKEN, server='demo')
con.is_connected()

In [ ]:
def New_Candle_Opened(Instrument, TimeFrame,LastOpen):
    CurrentBID_open = con.get_candles(instrument=Instrument, period=TimeFrame, number=1)['bidopen'][-1]
    if CurrentBID_open != LastOpen: 
        return True

In [ ]:
def Continuous_Trade(Instrument,TimeFrame,NextBar,StopLoss_pip,LOT,Reward_Risk_Ratio,NEXT_BUY_ENTRY,NEXT_SELL_ENTRY):
    global Entry
    Right_Minute_Time = int(datetime.now().strftime("%M"))
    if con.is_subscribed(Instrument) != True: con.subscribe_market_data(Instrument)
    while Right_Minute_Time != NextBar:  # Untill The candle closed
        Right_Minute_Time = int(datetime.now().strftime("%M"))
        Last_Bid = float(format(con.get_last_price(Instrument)[0], '.5f'))
        Last_Ask = float(format(con.get_last_price(Instrument)[1], '.5f'))
        Spread = Last_Ask - Last_Bid
        
        NEXT_BUY_ENTRY = Entry[0]
        NEXT_SELL_ENTRY = Entry[1]
        
        if Last_Ask >= NEXT_BUY_ENTRY:  # check number of orders too
            TP, SL = Generate_Limits('BUY',Last_Ask,StopLoss_pip,Reward_Risk_Ratio,Spread)
            BUY_Order = con.open_trade(Instrument,is_buy=True,amount=(LOT*100),order_type='AtMarket', 
                                       time_in_force='GTC',stop=SL,limit=TP,is_in_pips=False)
            Entry[0] = TP
            Entry[1] = SL
            print('NextBuy: ' + str(Entry[0]) + '\n' + 'NextSell: ' + str(Entry[1]))
            
        if Last_Bid <= NEXT_SELL_ENTRY:
            TP, SL = Generate_Limits('SELL',Last_Bid,StopLoss_pip,Reward_Risk_Ratio,Spread)
            SELL_Order = con.open_trade(Instrument,is_buy=False,amount=(LOT*100),order_type='AtMarket', 
                                       time_in_force='GTC',stop=SL,limit=TP,is_in_pips=False)
            Entry[1] = TP
            Entry[0] = SL
            print('NextBuy: ' + str(Entry[0]) + '\n' + 'NextSell: ' + str(Entry[1]))

In [ ]:
def Initial_Order_Creation(Instrument,TimeFrame,NextBar,StopLoss_pip,LOT,Reward_Risk_Ratio):  # RMT = Right Minute Time
    global Entry
    con.subscribe_market_data(Instrument)
    Last_Bid = float(format(con.get_last_price(Instrument)[0], '.5f'))
    Last_Ask = float(format(con.get_last_price(Instrument)[1], '.5f'))
    Spread = Last_Ask-Last_Bid
    
    BUY_TP, BUY_SL = Generate_Limits('BUY',Last_Ask,StopLoss_pip,Reward_Risk_Ratio,Spread)
    SELL_TP, SELL_SL = Generate_Limits('SELL',Last_Bid,StopLoss_pip,Reward_Risk_Ratio,Spread)
    Entry[0] = BUY_TP
    Entry[1] = BUY_SL
    
    BUY_Order = con.open_trade(Instrument,is_buy=True,amount=(LOT*100),order_type='AtMarket', 
                               time_in_force='GTC',stop=BUY_SL,limit=BUY_TP,is_in_pips=False)
    SELL_Order = con.open_trade(Instrument,is_buy=False,amount=(LOT*100),order_type='AtMarket', 
                               time_in_force='GTC',stop=SELL_SL,limit=SELL_TP,is_in_pips=False)
    Continuous_Trade(Instrument,TimeFrame,NextBar,StopLoss_pip,LOT,Reward_Risk_Ratio,BUY_TP,BUY_SL)
    

In [ ]:
def Generate_Limits(OrderType,OpenPrice,StopLoss_pip,Reward_Risk_Ratio,Spread):
    if OrderType == 'BUY':
        TakeProfit = float(format((OpenPrice  + (StopLoss_pip/100000)*Reward_Risk_Ratio),'.5f'))
        StopLoss = float(format((OpenPrice - Spread - (StopLoss_pip/100000)),'.5f'))
        return TakeProfit,StopLoss
    elif OrderType == 'SELL':
        TakeProfit = float(format((OpenPrice - (StopLoss_pip/100000)*Reward_Risk_Ratio) ,'.5f'))
        StopLoss = float(format((OpenPrice  + Spread + (StopLoss_pip/100000)),'.5f'))
        return TakeProfit,StopLoss

In [ ]:
def Next_Candle_Bartime(RMT,TimeFrame) :
    BarTime = int(TimeFrame[1:])
    Next_Open = RMT + BarTime
    if Next_Open == 60:
        Next_Open = 0
    return Next_Open

In [ ]:
TimeFrame = 'm5'
StopLoss_pip = 20;
Reward_Risk_Ratio = 1;
LOT = 0.01
BarTime = int(TimeFrame[1:])

Instrument = 'AUD/USD'

Entry = np.zeros(2)

while True:
    Right_Minute_Time = int(datetime.now().strftime("%M"))
    
    if Right_Minute_Time%BarTime == 0:
        df = con.get_open_positions(kind='dataframe')
        Position_Number = len(df)
        NextBar = Next_Candle_Bartime(Right_Minute_Time,TimeFrame)
        if Position_Number == 0:
            Initial_Order_Creation(Instrument,TimeFrame,NextBar,StopLoss_pip,LOT, Reward_Risk_Ratio)
            
        elif Position_Number <= 2:
            Entry[0] = df['limit'][0]
            Entry[1] = df['stop'][0]
            if Entry[0] < Entry[1] : 
                Entry[1] = df['limit'][0]
                Entry[0] = df['stop'][0]
            Continuous_Trade(Instrument,TimeFrame,NextBar,StopLoss_pip,LOT,Reward_Risk_Ratio,Entry[0],Entry[1])
            
       
       
    # tradeObj = thr.Thread(target = Initial_Order_Creation , args=(Instrument,StopLoss_pip,LOT, Reward_Risk_Ratio,Right_Minute_Time,BarTime))
       # tradeObj.start()
        print('Upper: ' + str(Entry[0]) + '\n' + 'Lower: ' + str(Entry[1]))
    